## Lab 04 Vector databases

# PG

### Creating db_url

In [1]:
from sqlalchemy.engine import URL

db_url = URL.create(
    drivername="postgresql+psycopg",
    username="postgres",
    password="password",
    host="localhost",
    port=5432,
    database="similarity_search_service_db",
)

### Models

In [2]:
from pgvector.sqlalchemy import Vector
from sqlalchemy import Integer, String
from typing import List
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column


class Base(DeclarativeBase):
    __abstract__ = True


class Image(Base):
    __tablename__ = "images"
    VECTOR_LENGTH = 512

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    image_path: Mapped[str] = mapped_column(String(256))
    image_embedding: Mapped[List[float]] = mapped_column(Vector(VECTOR_LENGTH))


### Connecting to db

In [3]:
from sqlalchemy import create_engine

engine = create_engine(db_url)

### Creating Table

In [4]:
Base.metadata.create_all(engine)

### Queries

In [5]:
from sqlalchemy.orm import Session
import sqlalchemy
import numpy as np


def insert_image(
    engine: sqlalchemy.Engine, image_path: str, image_embedding: list[float]
):
    with Session(engine) as session:
        image = Image(image_path=image_path, image_embedding=image_embedding)
        session.add(image)
        session.commit()


N = 100
for i in range(N):
    image_path = f"image_{i}.jpg"
    image_embedding = np.random.rand(512).tolist()
    insert_image(engine, image_path, image_embedding)

with Session(engine) as session:
    image = session.query(Image).first()


def find_k_images(
    engine: sqlalchemy.Engine, k: int, orginal_image: Image
) -> list[Image]:
    with Session(engine) as session:
        # execution_options={"prebuffer_rows": True} is used to prebuffer the rows, this is useful when we want to fetch the rows in chunks and return them after session is closed
        result = session.execute(
            sqlalchemy.select(Image)
            .order_by(
                Image.image_embedding.cosine_distance(orginal_image.image_embedding)
            )
            .limit(k),
            execution_options={"prebuffer_rows": True},
        )
        return result


k = 10
similar_images = find_k_images(engine, k, image)

In [6]:
list(similar_images)[0][0].image_path

'image_0.jpg'

### Filtering

In [7]:
from sqlalchemy import select


def find_images_with_similarity_score_greater_than(
    engine: sqlalchemy.Engine, similarity_score: float, orginal_image: Image
) -> list[Image]:
    with Session(engine) as session:
        result = session.execute(
            select(Image).filter(
                Image.image_embedding.cosine_distance(orginal_image.image_embedding)
                > similarity_score
            ),
            execution_options={"prebuffer_rows": True},
        )
        return result

In [8]:
img = Image(image_path="test_path.jpg", image_embedding=np.random.rand(512).tolist())

find_images_with_similarity_score_greater_than(
    engine, similarity_score=0.65, orginal_image=img
)

## Steam

In [9]:
from datasets import load_dataset

dataset = load_dataset("FronkonGames/steam-games-dataset")

columns = dataset["train"].features
print(columns)

columns_to_keep = [
    "Name",
    "Windows",
    "Linux",
    "Mac",
    "About the game",
    "Supported languages",
    "Price",
]

N = 40000
dataset = dataset["train"].select_columns(columns_to_keep).select(range(N))


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-e2ed184370a069(…):   0%|          | 0.00/123M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/83560 [00:00<?, ? examples/s]

{'AppID': Value('int64'), 'Name': Value('string'), 'Release date': Value('string'), 'Estimated owners': Value('string'), 'Peak CCU': Value('int64'), 'Required age': Value('int64'), 'Price': Value('float64'), 'DLC count': Value('int64'), 'About the game': Value('string'), 'Supported languages': Value('string'), 'Full audio languages': Value('string'), 'Reviews': Value('string'), 'Header image': Value('string'), 'Website': Value('string'), 'Support url': Value('string'), 'Support email': Value('string'), 'Windows': Value('bool'), 'Mac': Value('bool'), 'Linux': Value('bool'), 'Metacritic score': Value('int64'), 'Metacritic url': Value('string'), 'User score': Value('int64'), 'Positive': Value('int64'), 'Negative': Value('int64'), 'Score rank': Value('float64'), 'Achievements': Value('int64'), 'Recommendations': Value('int64'), 'Notes': Value('string'), 'Average playtime forever': Value('int64'), 'Average playtime two weeks': Value('int64'), 'Median playtime forever': Value('int64'), 'Medi

In [10]:
from sqlalchemy import Integer, Float, Boolean


class Games(Base):
    __tablename__ = "games"
    __table_args__ = {"extend_existing": True}

    # the vector size produced by the model taken from documentation https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2
    VECTOR_LENGTH = 512  # check the model output dimensionality

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String(256))
    description: Mapped[str] = mapped_column(String(4096))
    windows: Mapped[bool] = mapped_column(Boolean)
    linux: Mapped[bool] = mapped_column(Boolean)
    mac: Mapped[bool] = mapped_column(Boolean)
    price: Mapped[float] = mapped_column(Float)
    game_description_embedding: Mapped[List[float]] = mapped_column(
        Vector(VECTOR_LENGTH)
    )


Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

### Sentence transformers

In [11]:
from sentence_transformers import SentenceTransformer


checkpoint = "distiluse-base-multilingual-cased-v2"
model = SentenceTransformer(checkpoint, device="cpu")


def generate_embeddings(text: str) -> list[float]:
    return model.encode(text)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

### Insert games

In [12]:
from tqdm import tqdm


def insert_games(engine, dataset):
    with tqdm(total=len(dataset)) as pbar:
        for i, game in enumerate(dataset):
            game_description = game["About the game"] or ""
            game_embedding = generate_embeddings(game_description)
            name, windows, linux, mac, price = (
                game["Name"],
                game["Windows"],
                game["Linux"],
                game["Mac"],
                game["Price"],
            )
            if name and windows and linux and mac and price and game_description:
                game = Games(
                    name=game["Name"],
                    description=game_description[0:4096],
                    windows=game["Windows"],
                    linux=game["Linux"],
                    mac=game["Mac"],
                    price=game["Price"],
                    game_description_embedding=game_embedding,
                )
                with Session(engine) as session:
                    session.add(game)
                    session.commit()
            pbar.update(1)

In [ ]:
insert_games(engine, dataset)

100%|████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [1:03:02<00:00, 10.58it/s]


### Find games

In [14]:
from typing import Optional


def find_game(
    engine: sqlalchemy.Engine,
    game_description: str,
    windows: Optional[bool] = None,
    linux: Optional[bool] = None,
    mac: Optional[bool] = None,
    price: Optional[int] = None,
):
    with Session(engine) as session:
        game_embedding = generate_embeddings(game_description)

        query = select(Games).order_by(
            Games.game_description_embedding.cosine_distance(game_embedding)
        )

        if price:
            query = query.filter(Games.price <= price)
        if windows:
            query = query.filter(Games.windows)
        if linux:
            query = query.filter(Games.linux)
        if mac:
            query = query.filter(Games.mac)

        result = session.execute(query, execution_options={"prebuffer_rows": True})
        game = result.scalars().first()

        return game

In [15]:
game = find_game(engine, "This is a game about a hero who saves the world", price=10)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

game = find_game(engine, game_description="Home decorating", price=20)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

Game: Ultimate Spider Hero
Description: Ultimate Spider Hero game was designed for real heroes! Your mission is to help poor residents of the Metropolis and to save them from the terrible monsters. Move forward to fight your enemies and try not to fall! Features: Simple and addictive gameplay Nice graphics Awesome Ultimate Spider Hero Countless Steam achievements for you to collect! Compatibility with multiple major platforms (Windows, Mac, Linux, SteamOS) Make your way through the endless labyrinths of long, confusing city streets together with your favorite hero from countless movies and cartoons! Although this may look simple enough, things are not as easy as they seem. You will have to learn how to cling into houses properly using your web, otherwise you will fall to your demise. If you manage to do so - you will become a real superhero, armed with elusiveness, agility and speed and the ability to tirelessly swing across the rooftops and between the huge skyscrapers this urban land

In [16]:
game = find_game(engine, game_description="Home decorating", mac=True, price=5)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

Game: 3D PUZZLE - Old House
Description: Collect a 3D puzzle, transferring things to the right places to create a beautiful house. You need to go to the item, take it by pressing the left mouse button and take the item to the desired location marked in green. If you brought the correct item, it will snap into place and you will receive leaderboard points and achievements for this. Collect as much substance as possible as quickly as possible to get more points for the leaderboard. If you brought the wrong item, you can throw it away, it will return to the starting location so that you can pick it up again.


# RAG

### Connec to to milvus

In [ ]:
from pymilvus import MilvusClient

host = "localhost"
port = "19530"

milvus_client = MilvusClient(host=host, port=port)

### Create schema

In [ ]:
from pymilvus import FieldSchema, DataType, CollectionSchema

VECTOR_LENGTH = 768  # check the dimensionality for Silver Retriever Base (v1.1) model

id_field = FieldSchema(
    name="id", dtype=DataType.INT64, is_primary=True, description="Primary id"
)
text = FieldSchema(
    name="text", dtype=DataType.VARCHAR, max_length=4096, description="Page text"
)
embedding_text = FieldSchema(
    "embedding",
    dtype=DataType.FLOAT_VECTOR,
    dim=VECTOR_LENGTH,
    description="Embedded text",
)

fields = [id_field, text, embedding_text]

schema = CollectionSchema(
    fields=fields,
    auto_id=True,
    enable_dynamic_field=True,
    description="RAG Texts collection",
)


### Create collection

In [ ]:
COLLECTION_NAME = "rag_texts_and_embeddings"

milvus_client.create_collection(collection_name=COLLECTION_NAME, schema=schema)

index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="embedding",
    index_type="HNSW",
    metric_type="L2",
    params={"M": 4, "efConstruction": 64},  # lower values for speed
)


milvus_client.create_index(collection_name=COLLECTION_NAME, index_params=index_params)

# checkout our collection
print(milvus_client.list_collections())

# describe our collection
print(milvus_client.describe_collection(COLLECTION_NAME))

['rag_texts_and_embeddings']
{'collection_name': 'rag_texts_and_embeddings', 'auto_id': True, 'num_shards': 1, 'description': 'RAG Texts collection', 'fields': [{'field_id': 100, 'name': 'id', 'description': 'Primary id', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'text', 'description': 'Page text', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4096}}, {'field_id': 102, 'name': 'embedding', 'description': 'Embedded text', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'functions': [], 'aliases': [], 'collection_id': 461835363952623831, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True, 'created_timestamp': 461835374861484038}


### Constants

In [ ]:
# define data source and destination
## the document origin destination from which document will be downloaded
pdf_url = "https://www.iab.org.pl/wp-content/uploads/2024/04/Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the document
file_name = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the processed document
file_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.json"

## local destination of the embedded pages of the document
embeddings_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska-Embeddings.json"

## local destination of all above local required files
data_dir = "./data"

### Download data

In [ ]:
import os
import requests


def download_pdf_data(pdf_url: str, file_name: str) -> None:
    response = requests.get(pdf_url, stream=True)
    with open(os.path.join(data_dir, file_name), "wb") as file:
        for block in response.iter_content(chunk_size=1024):
            if block:
                file.write(block)


download_pdf_data(pdf_url, file_name)

### Preparing and chunking

In [ ]:
import fitz
import json


def extract_pdf_text(file_name, file_json):
    document = fitz.open(os.path.join(data_dir, file_name))
    pages = []

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        page_text = page.get_text()
        pages.append({"page_num": page_num, "text": page_text})

    with open(os.path.join(data_dir, file_json), "w") as file:
        json.dump(pages, file, indent=4, ensure_ascii=False)


extract_pdf_text(file_name, file_json)

### Vectorize data

In [ ]:
from sentence_transformers import SentenceTransformer


def generate_embeddings(file_json, embeddings_json, model):
    pages = []
    with open(os.path.join(data_dir, file_json), "r") as file:
        data = json.load(file)

    for page in data:
        pages.append(page["text"])

    embeddings = model.encode(pages)

    embeddings_paginated = []
    for page_num in range(len(embeddings)):
        embeddings_paginated.append(
            {"page_num": page_num, "embedding": embeddings[page_num].tolist()}
        )

    with open(os.path.join(data_dir, embeddings_json), "w") as file:
        json.dump(embeddings_paginated, file, indent=4, ensure_ascii=False)


model_name = "ipipan/silver-retriever-base-v1.1"
device = "cpu"
model = SentenceTransformer(model_name, device=device)
generate_embeddings(file_json, embeddings_json, model)

### Insert embeddings

In [57]:
def insert_embeddings(
    file_json,
    embeddings_json,
    client=milvus_client,
    collection_name="rag_texts_and_embeddings",
):
    rows = []
    with (
        open(os.path.join(data_dir, file_json), "r") as t_f,
        open(os.path.join(data_dir, embeddings_json), "r") as e_f,
    ):
        text_data, embedding_data = json.load(t_f), json.load(e_f)
        text_data = list(map(lambda d: d["text"], text_data))
        embedding_data = list(map(lambda d: d["embedding"], embedding_data))

        for page, (text, embedding) in enumerate(zip(text_data, embedding_data)):
            rows.append({"text": text, "embedding": embedding})

    client.insert(collection_name=collection_name, data=rows)

In [ ]:
insert_embeddings(file_json, embeddings_json)

# load inserted data into memory
milvus_client.load_collection("rag_texts_and_embeddings")

### Search

In [71]:
def search(
    model,
    query,
    client=milvus_client,
    collection_name="rag_texts_and_embeddings",
    metric_type="L2",
):
    embedded_query = model.encode(query).tolist()
    result = client.search(
        collection_name=collection_name,
        data=[embedded_query],
        limit=1,
        search_params={"metric_type": metric_type},
        output_fields=["text"],
    )
    return result

In [ ]:
result = search(model, query="Czym jest sztuczna inteligencja")
print(result[0][0]["text"])

Historia powstania
sztucznej inteligencji
7
W języku potocznym „sztuczny" oznacza to, co
jest 
wytworem 
mającym 
naśladować 
coś
naturalnego. W takim znaczeniu używamy
terminu ,,sztuczny'', gdy mówimy o sztucznym
lodowisku lub oku. Sztuczna inteligencja byłaby
czymś (programem, maszyną) symulującym
inteligencję naturalną, ludzką.
Sztuczna inteligencja (AI) to obszar informatyki,
który skupia się na tworzeniu programów
komputerowych zdolnych do wykonywania
zadań, które wymagają ludzkiej inteligencji. 
Te zadania obejmują rozpoznawanie wzorców,
rozumienie języka naturalnego, podejmowanie
decyzji, uczenie się, planowanie i wiele innych.
Głównym celem AI jest stworzenie systemów,
które są zdolne do myślenia i podejmowania
decyzji na sposób przypominający ludzki.
Historia sztucznej inteligencji sięga lat 50. 
XX wieku, kiedy to powstały pierwsze koncepcje
i modele tego, co mogłoby stać się sztuczną
inteligencją. Jednym z pionierów był Alan
Turing, który sformułował test Turinga, mający
na 

### Gemini API integration

In [43]:
GEMINI_KEY = os.getenv("GEMINI_API_KEY")

In [45]:
from google import genai

GEMINI_KEY = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=GEMINI_KEY)

MODEL = "gemini-2.5-flash"


def generate_response(prompt: str):
    try:
        response = gemini_client.models.generate_content(
            model=MODEL,
            contents=prompt,
        )
        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [72]:
# Prompt is in polish, because the context is in polish


def build_prompt(context: str, query: str) -> str:
    prompt = f"""
        Jesteś ekspertem w dziedzinie Sztucznej Inteligencji i Uczenia Maszynowego.
        Na podstawie dostarczonego kontekstu:

        {context}

        Znajdź odpowiedź na następujące pytanie:
        
        {query}

        Bądź bardzo precyzyjny i nie podawaj żadnych informacji, które nie znajdują się w dostarczonym kontekście.
        Jeśli w kontekście nie ma odpowiedzi, odpowiedz "Nie znaleziono informacji".
    """
    return prompt


def rag(
    model,
    query: str,
    collection_nane: str = "rag_texts_and_embeddings",
    metric_type: str = "L2",
) -> str:
    context = search(
        model, query=query, collection_name=collection_nane, metric_type=metric_type
    )[0][0]["text"]
    prompt = build_prompt(context, query)
    response = generate_response(prompt)

    return response

### Having fun with RAG

In [50]:
query1 = "Czym jest Contrastive Language-Image Pretraining?"
response1 = rag(model, query1)
print(response1)

Nie znaleziono informacji.


#### Debug why 'nie znaleziono informacji'

In [ ]:
context = search(model, query=query1)[0][0]["text"]
print(context)


Wielkość zasobów internetowych, na podstawie
których LLM rozwija swoje możliwości, staje się
coraz większa, a nad ich jakością czuwają
twórcy modelu. Przełomem w ustanawianiu
standardów z zakresu modelowania języka był
ChatGPT-2, który pokazał, jak zwiększenie skali
modelu może prowadzić do nowych i bardziej
zaawansowanych możliwości. Te umiejętności
tworzenia tekstu przez GPT-2 przełożyły się na
zdolność do wytwarzania długich, spójnych i
kontekstowo poprawnych treści, z tworzeniem
esejów, opowiadań, a nawet poezji włącznie.
Model wykazał też zdolność do rozumienia i
rozwiązywania 
bardziej 
złożonych 
zadań
językowych, 
takich 
jak 
rozróżnianie
dwuznaczności 
w 
tekstach. 
Kolejne 
etapy
rozwoju popularnego czatu doprowadziły do
wersji 
GPT-4, 
wykraczającej 
już 
poza
modelowanie językowe i obejmującej multi-
media.
Jeśli zastanawiasz się, dlaczego ciągle mówimy
o "przewidywaniu jednego, kolejnego słowa",
podczas gdy na przykład czat GPT bez trudu
generuje długie teksty, to odpowie

Context is wrong, however there is information about the query in doc.

Let's try with easier query.

In [ ]:
query2 = (
    "Jaki model jest dobry do generowania sztuki, np. do materiałów marketingowych?"
)
response2 = rag(model, query2)
print(response2)

Generatywne modele grafiki komputerowej, w tym Pix2Pix, są dobre do generowania sztuki, projektowania i produkcji treści multimedialnych.


For that query, model halucinated. The answer should be Blue Willow.

In [54]:
context2 = search(model, query=query2)[0][0]["text"]
print(context2)

Inne modele sztucznej
inteligencji
1 6
Generatywne modele grafiki
komputerowej:
Pix2Pix: Ten model jest wykorzystywany do
konwersji obrazów z jednej dziedziny na obrazy
w innej. Na przykład może przekształcać obrazy
czarnobiałe na kolorowe, rysunki w realistyczne
obrazy, itp.
Sztuczna 
inteligencja 
generatywna 
ma
potencjał zastosowania w wielu dziedzinach,
takich jak: sztuka, projektowanie, produkcja
treści multimedialnych, czy nawet medycyna.
Jednakże, 
równocześnie 
z 
zastosowaniami
pozytywnymi, pojawiają się także wyzwania
związane z etyką i bezpieczeństwem, zwłaszcza
w 
kontekście 
fałszywych 
informacji 
i
deepfake'ów.
Sztuczna inteligencja dedukcyjna:
Skoncentrowana na wykorzystywaniu reguł
logicznych 
do 
wyciągania 
wniosków 
i
podejmowania decyzji. Ten rodzaj AI działa na
podstawie dostarczonych danych i reguł, nie
wymaga eksploracyjnego uczenia się.
Sztuczna inteligencja indukcyjna: 
Oparta na zdolności do wyciągania ogólnych
zasad lub wniosków na podstawie konkretnych
prz

Again, wrong context were parsed. I assume this is caused because of metric_type="L2", which is very bad for high dimensional spaces (in this case 784).


## Second try with cosine metric

Create similar collection to above but with cosine_similarity metric_type and try search for the same queries again.

In [56]:
COLLECTION_NAME_COSINE = "rag_texts_and_embeddings_cosine"

milvus_client.create_collection(collection_name=COLLECTION_NAME_COSINE, schema=schema)

index_params_cosine = milvus_client.prepare_index_params()

index_params_cosine.add_index(
    field_name="embedding",
    index_type="HNSW",
    metric_type="COSINE",
    params={"M": 4, "efConstruction": 64},  # lower values for speed
)


milvus_client.create_index(
    collection_name=COLLECTION_NAME_COSINE, index_params=index_params_cosine
)

# checkout our collection
print(milvus_client.list_collections())

# describe our collection
print(milvus_client.describe_collection(COLLECTION_NAME_COSINE))

['rag_texts_and_embeddings_cosine', 'rag_texts_and_embeddings']
{'collection_name': 'rag_texts_and_embeddings_cosine', 'auto_id': True, 'num_shards': 1, 'description': 'RAG Texts collection', 'fields': [{'field_id': 100, 'name': 'id', 'description': 'Primary id', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'text', 'description': 'Page text', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4096}}, {'field_id': 102, 'name': 'embedding', 'description': 'Embedded text', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'functions': [], 'aliases': [], 'collection_id': 461835363954033315, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True, 'created_timestamp': 461835861047115779}


In [60]:
insert_embeddings(file_json, embeddings_json, collection_name=COLLECTION_NAME_COSINE)
milvus_client.load_collection(COLLECTION_NAME_COSINE)

### Experiments

In [73]:
query3 = "Czym jest Contrastive Language-Image Pretraining?"
response3 = rag(
    model, query3, collection_nane=COLLECTION_NAME_COSINE, metric_type="COSINE"
)
print(response3)

Nie znaleziono informacji.


In [74]:
contxet3 = search(
    model, query3, collection_name=COLLECTION_NAME_COSINE, metric_type="COSINE"
)

print(contxet3[0][0]["text"])

Wielkość zasobów internetowych, na podstawie
których LLM rozwija swoje możliwości, staje się
coraz większa, a nad ich jakością czuwają
twórcy modelu. Przełomem w ustanawianiu
standardów z zakresu modelowania języka był
ChatGPT-2, który pokazał, jak zwiększenie skali
modelu może prowadzić do nowych i bardziej
zaawansowanych możliwości. Te umiejętności
tworzenia tekstu przez GPT-2 przełożyły się na
zdolność do wytwarzania długich, spójnych i
kontekstowo poprawnych treści, z tworzeniem
esejów, opowiadań, a nawet poezji włącznie.
Model wykazał też zdolność do rozumienia i
rozwiązywania 
bardziej 
złożonych 
zadań
językowych, 
takich 
jak 
rozróżnianie
dwuznaczności 
w 
tekstach. 
Kolejne 
etapy
rozwoju popularnego czatu doprowadziły do
wersji 
GPT-4, 
wykraczającej 
już 
poza
modelowanie językowe i obejmującej multi-
media.
Jeśli zastanawiasz się, dlaczego ciągle mówimy
o "przewidywaniu jednego, kolejnego słowa",
podczas gdy na przykład czat GPT bez trudu
generuje długie teksty, to odpowie

Again same issue.

In [75]:
query4 = (
    "Jaki model jest dobry do generowania sztuki, np. do materiałów marketingowych?"
)
response4 = rag(
    model, query4, collection_nane=COLLECTION_NAME_COSINE, metric_type="COSINE"
)
print(response4)

Na podstawie dostarczonego kontekstu, dobrym modelem do generowania sztuki, np. do materiałów marketingowych, jest **Pix2Pix**, należący do kategorii generatywnych modeli grafiki komputerowej. W kontekście wspomniano, że sztuczna inteligencja generatywna ma potencjał zastosowania w sztuce, projektowaniu i produkcji treści multimedialnych.


Again...

In [76]:
query5 = "O co chodzi w pierwszym etapie, czyli preprodukcji?. Co ona zawiera??"
response5 = rag(
    model, query5, collection_nane=COLLECTION_NAME_COSINE, metric_type="COSINE"
)
print(response5)

Pierwszy etap, czyli preprodukcja, to faza planowania. Obejmuje ona:

1.  Zdefiniowanie celu, pomysłu, koncepcji.
2.  Zaplanowanie budżetu.
3.  Podjęcie decyzji nt. metody produkcji, rodzaju wideo lub gatunku filmowego.

Na zakończenie tego etapu powinno się dysponować scenariuszem, scenopisem, harmonogramem, zdefiniowanym budżetem i wybranymi narzędziami potrzebnymi do produkcji i postprodukcji.


In [77]:
query5 = "O co chodzi w pierwszym etapie, czyli preprodukcji?. Co ona zawiera??"
response5 = rag(model, query5)
print(response5)

Pierwszy etap, czyli preprodukcja, to faza planowania. Obejmuje ona:

1.  Zdefiniowanie celu, pomysłu, koncepcji.
2.  Zaplanowanie budżetu.
3.  Podjęcie decyzji dotyczących metody produkcji, rodzaju wideo lub gatunku filmowego.

Na zakończenie tego etapu powinno się dysponować scenariuszem, scenopisem, harmonogramem, zdefiniowanym budżetem i wybranymi narzędziami potrzebnymi do produkcji i postprodukcji.


With very specified question it was finally able to find answer for the question.

Overall: RAG is hard.